In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np

from scipy import stats
from statsmodels.stats.multitest import multipletests
from scipy.signal import find_peaks

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pyarrow.feather as feather


import matplotlib.pyplot as plt
import seaborn as sns


import importlib

import holoviews as hv

hv.extension("bokeh")

In [ ]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [ ]:
# Load already existing dataset: 

Subset = feather.read_feather("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Learning/240809_BallposData.feather")

In [ ]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*Learning*")]

folders

In [ ]:
#Generate the experiments

Exps = [Ballpushing_utils.Experiment(f) for f in folders]

In [ ]:
Data = Ballpushing_utils.Dataset(Exps)

In [ ]:
Data.generate_dataset(success_cutoff=False)

In [ ]:
# Get the data columns
  
Data.data.columns

In [ ]:
# Make a subset of the data keeping only yball, yball_smooth,

Subset = Data.data[["Frame","time", "yball", "yball_smooth", "yball_relative", "fly", "Peak", "Date"]]

# Save this subset to a feather file

#feather.write_feather(Subset, "/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Learning/240809_BallposData.feather")

In [ ]:
Subset

In [ ]:
# Get the name of the first fly and store it to use it for the next steps

fly = Subset["fly"].iloc[0]

TestData = Subset[Subset["fly"] == fly]

# Now take one fly and plot the yball_relative

Subset[Subset["fly"] == fly].hvplot(x="time", y="yball_relative", kind="scatter")

In [ ]:
# Compute the derivative of the yball_relative

Subset["yball_relative_derivative"] = Subset["yball_relative"].diff()

In [ ]:


# Plot the derivative

Subset[Subset["fly"] == fly].hvplot(x="time", y="yball_relative_derivative", kind="scatter")



In [ ]:
# Do Negative peak detection on the derivative

Peaks = find_peaks(-TestData["yball_relative_derivative"], height=0.23, distance=500)

Peaks

In [ ]:
# Extract the "time" and "Frame" columns from the dataset
frame_to_time_mapping = dict(zip(TestData["Frame"], TestData["time"]))

# Convert Peaks data to lists
x_peaks = Peaks[0].tolist()
y_peaks = Peaks[1]["peak_heights"].tolist()

# Map frame indices to time values
x_peaks_time = [frame_to_time_mapping[frame] for frame in x_peaks]

# Plot the derivative and the peaks
scatter_plot = TestData.hvplot(x="time", y="yball_relative_derivative", kind="scatter")
peaks_plot = hv.Scatter(
    (x_peaks_time, y_peaks), kdims=["time"], vdims=["peak_heights"], label="Peaks"
)

# Combine the plots
combined_plot = scatter_plot * peaks_plot
combined_plot

In [ ]:
import pandas as pd
from scipy.signal import find_peaks

# Group by individual flies
grouped = Subset.groupby("fly")


# Function to process each group
def process_group(group):
    # Find peaks in the derivative
    peaks, _ = find_peaks(
        -group["yball_relative_derivative"], height=0.3, distance=500
    )

    # Debug: Print the peaks detected for each fly
    print(f"Fly: {group['fly'].iloc[0]}, Peaks: {peaks}")

    # Initialize the "Trial" column
    group["Trial"] = 0

    # Assign trial numbers based on peak positions
    trial_number = 1
    previous_peak = 0
    for peak in peaks:
        group.iloc[previous_peak : peak + 1, group.columns.get_loc("Trial")] = (
            trial_number
        )
        trial_number += 1
        previous_peak = peak + 1

    # Assign the last trial number to the remaining rows
    group.iloc[previous_peak:, group.columns.get_loc("Trial")] = trial_number

    # Debug: Print the trial numbers assigned for each fly
    print(group[["fly", "Frame", "Trial"]].head(10))

    return group


# Apply the function to each group and combine the results
Trials = grouped.apply(process_group).reset_index(drop=True)

In [ ]:
# Find how many unique trials there are grouped by fly
Trials.groupby("fly")["Trial"].nunique()

In [ ]:
# Now we can drop any fly that has less than 2 trials

Filtered = Trials.groupby("fly").filter(lambda x: x["Trial"].nunique() > 1)

Filtered

In [ ]:
Filtered.groupby("fly")["Trial"].nunique()

In [ ]:
# Group by Trial and time, then compute the mean of yball_relative
averaged_data = (
    Filtered.groupby(["Trial", "time"])["yball_relative"].mean().reset_index()
)

# Plot the averaged yball_relative values
averaged_data.hvplot(
    x="time", y="yball_relative", by="Trial", kind="line", legend="top_left"
)

In [ ]:
# Among the flies that have more than 1 trial, we need to drop the Frames that belong to a plateau, meaning the frames where the yball_relative is above a certain threshold


# Function to clean each trial
def clean_trial(group):
    # Drop the first 500 frames
    group = group.iloc[500:]

    # Find the index where yball_relative reaches its max value for the first time
    max_index = group["yball_relative"].idxmax()

    # Trim the trial data to end at this maximum value
    group = group.loc[:max_index]

    return group


# Apply the function to each group and combine the results
cleaned_data = (
    Filtered.groupby(["fly", "Trial"]).apply(clean_trial).reset_index(drop=True)
)

# Display the cleaned dataset
cleaned_data

In [ ]:
# Function to compute trial duration
def compute_trial_duration(group):
    duration = group["time"].max() - group["time"].min()
    return pd.Series({"duration": duration})


# Apply the function to each group and compute the trial durations
trial_durations = (
    cleaned_data.groupby(["fly", "Trial"]).apply(compute_trial_duration).reset_index()
)

In [ ]:
Jitterbox = HoloviewsTemplates.jitter_boxplot(trial_durations, kdims = "Trial", metric="duration", plot_options=HoloviewsTemplates.hv_slides).opts(invert_axes=False, xlabel="Trial Number", ylabel="Duration (s)")

In [ ]:
Jitterbox

In [ ]:
# Save the plot as a PNG and a SVG file
hv.save(
    Jitterbox,
    "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Learning/240809_TrialDuration.png",
)

In [ ]:
# Assuming trial_durations is your DataFrame
# Ensure 'Trial' is treated as a categorical variable with ordered categories
trial_durations["Trial"] = pd.Categorical(trial_durations["Trial"], ordered=True)

# Create the jitterboxplot
jitterbox = HoloviewsTemplates.jitter_boxplot(
    trial_durations,
    kdims="Trial",
    metric="duration",
    plot_options=HoloviewsTemplates.hv_slides,
).opts(invert_axes=False, xlabel="Trial Number", ylabel="Duration (s)")

# Create a line plot to track each "fly" across trial numbers
line_plot = (
    hv.Curve(trial_durations, kdims=["Trial"], vdims=["duration", "fly"])
    .groupby("fly")
    .opts(color="gray", line_width=1, alpha=0.5)
    .overlay()
)

# Overlay the line plot on top of the jitterboxplot
combined_plot = (jitterbox * line_plot).opts(width = 1500,
                                             height = 1000,
                                             )

combined_plot

In [ ]:
import pandas as pd

# Assuming trial_durations is your DataFrame
# Ensure 'Trial' is treated as a categorical variable with ordered categories
trial_durations["Trial"] = pd.Categorical(trial_durations["Trial"], ordered=True)


# Function to calculate improvement
def calculate_improvement(df):
    df = df.sort_values(by="Trial")
    df["improved"] = df["duration"].diff().lt(0)
    return df


# Apply the function to each group of 'fly'
trial_durations = trial_durations.groupby("fly").apply(calculate_improvement)

# Calculate the proportion of flies that improved in each trial
improvement_proportions = trial_durations.groupby("Trial")["improved"].mean()

# Print the results
print(improvement_proportions)

# Plot the results
import holoviews as hv

hv.extension("bokeh")

improvement_plot = hv.Curve(improvement_proportions).opts(
    xlabel="Trial Number",
    ylabel="Proportion of Flies Improved",
    title="Proportion of Flies that Improved in Each Trial",
    width=800,
    height=400,
)

improvement_plot

In [ ]:
# Save the plot
hv.save(improvement_plot, "improvement_plot.html")

In [ ]:
# Subset to get only the "morning" data
morning_data = cleaned_data[cleaned_data["Peak"] == "morning"]

# Apply the function to each group and compute the trial durations
trial_durations = (
    morning_data.groupby(["fly", "Trial"]).apply(compute_trial_duration).reset_index()
)

# Redo the plot with the morning data

# Create the jitterboxplot
jitterbox = HoloviewsTemplates.jitter_boxplot(
    trial_durations,
    kdims="Trial",
    metric="duration",
    plot_options=HoloviewsTemplates.hv_slides,
).opts(invert_axes=False, xlabel="Trial Number", ylabel="Duration (s)")

# Create a line plot to track each "fly" across trial numbers
line_plot = (
    hv.Curve(trial_durations, kdims=["Trial"], vdims=["duration", "fly"])
    .groupby("fly")
    .overlay()
)

# Overlay the line plot on top of the jitterboxplot
combined_plot = (jitterbox * line_plot).opts(width = 1500,
                                             height = 1000,
                                             )

combined_plot

In [ ]:
# For flies for which "Peak" is NaN, give them the value "morning"
Filtered["Peak"] = Filtered["Peak"].fillna("morning")
cleaned_data["Peak"] = cleaned_data["Peak"].fillna("morning")

# Ensure 'Trial' is treated as a categorical variable with ordered categories
cleaned_data["Trial"] = pd.Categorical(cleaned_data["Trial"], ordered=True)


# Apply the function to each group and compute the trial durations
trial_durations_peak = (
    cleaned_data.groupby(["Peak","fly", "Trial"]).apply(compute_trial_duration).reset_index()
)


Jitterbox_peak = HoloviewsTemplates.jitter_boxplot(
    trial_durations_peak,
    kdims="Trial",
    metric="duration",
    plot_options=HoloviewsTemplates.hv_slides,
    groupby="Peak",
    render="grouped"
).opts(invert_axes=False, xlabel="Trial Number", ylabel="Duration (s)")

Jitterbox_peak

In [ ]:
import holoviews as hv
import pandas as pd

# For flies for which "Peak" is NaN, give them the value "morning"
Filtered["Peak"] = Filtered["Peak"].fillna("morning")
cleaned_data["Peak"] = cleaned_data["Peak"].fillna("morning")

# Ensure 'Trial' is treated as a categorical variable with ordered categories
cleaned_data["Trial"] = pd.Categorical(cleaned_data["Trial"], ordered=True)

# Apply the function to each group and compute the trial durations
trial_durations_peak = (
    cleaned_data.groupby(["Peak", "fly", "Trial"])
    .apply(compute_trial_duration)
    .reset_index()
)

# Create the jitterboxplot
jitterbox_peak = HoloviewsTemplates.jitter_boxplot(
    trial_durations_peak,
    kdims="Trial",
    metric="duration",
    plot_options=HoloviewsTemplates.hv_slides,
    groupby="Peak",
    render="grouped",
).opts(invert_axes=False, xlabel="Trial Number", ylabel="Duration (s)")

# Create a line plot to track each "fly" across trial numbers
line_plot_peak = (
    hv.Curve(trial_durations_peak, kdims=["Trial", "Peak"], vdims=["duration", "fly"])
    .groupby(["Peak", "fly"])
    .overlay()
)

# Overlay the line plot on top of the jitterboxplot
combined_plot_peak = (jitterbox_peak * line_plot_peak).opts(
    width=1500,
    height=1000,
)

combined_plot_peak

In [ ]:
# Save the plot as a PNG and a SVG file
hv.save(Jitterbox, "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Learning/240809_TrialDuration.png")


In [ ]:
from bokeh.io.export import export_svgs

def export_svg(obj, filename):
    plot_state = hv.renderer("bokeh").get_plot(obj).state
    plot_state.output_backend = "svg"
    export_svgs(plot_state, filename=filename)


export_svg(Jitterbox, "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/Learning/240809_TrialDuration.svg")

In [ ]:
# Convert Trial back to integers
cleaned_data["Trial"] = cleaned_data["Trial"].astype(int)

# Function to compute the end time of each trial
def compute_trial_end_time(group):
    end_time = group["time"].max()
    return pd.Series({"end_time": end_time})


# Apply the function to each group and compute the end times
trial_end_times = (
    cleaned_data.groupby(["fly", "Trial"]).apply(compute_trial_end_time).reset_index()
)

# Sort by end_time
trial_end_times = trial_end_times.sort_values(by="end_time")

# Compute the cumulative count of solved trials
trial_end_times["cumulative_solved_trials"] = (
    trial_end_times["end_time"].rank(method="first").astype(int)
)

# Plot the cumulative count of solved trials over time
trial_end_times.hvplot.step(
    x="end_time",
    y="cumulative_solved_trials",
    title="Cumulative Solved Trials Over Time",
    xlabel="Time",
    ylabel="Cumulative Solved Trials",
)

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy import stats
import holoviews as hv
import hvplot.pandas
from holoviews import opts  # Import opts from holoviews


# Define logistic function
def logistic(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))


# Filter data to include only relevant subset
GroupData_morning = trial_end_times

# Calculate the number of unique 'flies' in the dataset
num_replicates = GroupData_morning["fly"].nunique()

# Initial guesses for L, k, x0
p0 = [
    max(GroupData_morning["cumulative_solved_trials"]),
    1,
    np.median(GroupData_morning["end_time"]),
]

# Fit logistic function to data
params, _ = curve_fit(
    logistic,
    GroupData_morning["end_time"],
    GroupData_morning["cumulative_solved_trials"],
    p0,
    maxfev=10000,
)

# params contains the fitted values for L, k, x0
L, k, x0 = params

# Create a new DataFrame for the logistic fit curve
logistic_fit = pd.DataFrame(
    {
        "end_time": np.linspace(
            GroupData_morning["end_time"].min(),
            GroupData_morning["end_time"].max(),
            100,
        ),
    }
)

# Calculate y values for the logistic fit curve
logistic_fit["cumulative_solved_trials"] = logistic(logistic_fit["end_time"], L, k, x0)

# Calculate R-squared for logistic fit curve
y_pred_logistic = logistic(GroupData_morning["end_time"], L, k, x0)
r_squared_logistic = r2_score(
    GroupData_morning["cumulative_solved_trials"], y_pred_logistic
)

# Calculate linear fit for the first half of the data
first_half = GroupData_morning.iloc[: len(GroupData_morning) // 2]
slope, intercept, _, _, _ = stats.linregress(
    first_half["end_time"], first_half["cumulative_solved_trials"]
)

# Create a new DataFrame for the linear fit line
linear_fit = pd.DataFrame(
    {
        "end_time": np.linspace(
            GroupData_morning["end_time"].min(),
            GroupData_morning["end_time"].max(),
            100,
        ),
    }
)

# Calculate y values for the linear fit line
linear_fit["cumulative_solved_trials"] = slope * linear_fit["end_time"] + intercept

# Calculate R-squared for linear fit curve
y_pred_linear = slope * GroupData_morning["end_time"] + intercept
r_squared_linear = r2_score(
    GroupData_morning["cumulative_solved_trials"], y_pred_linear
)

# Calculate center x value and min and maximum xy value
center_x = (
    GroupData_morning["end_time"].max() - GroupData_morning["end_time"].min()
) / 2
max_y = max(
    max(GroupData_morning["cumulative_solved_trials"]),
    max(logistic_fit["cumulative_solved_trials"]),
)

max_x = GroupData_morning["end_time"].max()
min_y = GroupData_morning["cumulative_solved_trials"].min()

# Create your plot using GroupData_morning and add the fits and annotations
cumulcurve_pool = (
    hv.Curve(
        data=GroupData_morning,
        kdims=["end_time"],
        vdims=["cumulative_solved_trials"],
    )
    .opts(
        height=1000,
        width=1200,
        alpha=1,
        line_width=2,
        xlabel="Time(s)",
        ylabel="Cumulative Solved Trials",
        show_grid=True,
        fontscale=3,
        title="Cumulative Solved Trials Over Time",
    )
    * hv.Curve(
        data=logistic_fit, kdims=["end_time"], vdims=["cumulative_solved_trials"]
    ).opts(color="green")
    * hv.Curve(
        data=linear_fit, kdims=["end_time"], vdims=["cumulative_solved_trials"]
    ).opts(color="red")
    # Uncomment the following lines to add text annotations
    * hv.Text(center_x, max_y - 7, f"Logistic fit R-squared: {r_squared_logistic:.2f}").opts(text_color="green")
    * hv.Text(center_x, max_y, f"Linear fit R-squared: {r_squared_linear:.2f}").opts(text_color="red")
    * hv.Text(max_x - 100, min_y * num_replicates, f"N = {num_replicates}")
).opts(
    opts.Area(
        show_legend=False,
        show_frame=False,
        fill_color="blue",
        line_color="black",
        line_width=0,
    )
)

cumulcurve_pool

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import holoviews as hv
import hvplot.pandas
from holoviews import opts


# Define double sigmoid function
def double_sigmoid(x, L1, k1, x01, L2, k2, x02):
    return (L1 / (1 + np.exp(-k1 * (x - x01)))) + (L2 / (1 + np.exp(-k2 * (x - x02))))


# Improved initial guesses for L1, k1, x01, L2, k2, x02
p0 = [
    max(GroupData_morning["cumulative_solved_trials"]) / 2,
    0.1,
    np.median(GroupData_morning["end_time"]) / 2,
    max(GroupData_morning["cumulative_solved_trials"]) / 2,
    0.1,
    np.median(GroupData_morning["end_time"]) * 1.5,
]

# Increase maxfev
maxfev_value = 10000

# Fit double sigmoid function to data
params, _ = curve_fit(
    double_sigmoid,
    GroupData_morning["end_time"],
    GroupData_morning["cumulative_solved_trials"],
    p0,
    maxfev=maxfev_value,
)

# params contains the fitted values for L1, k1, x01, L2, k2, x02
L1, k1, x01, L2, k2, x02 = params

# Calculate fitted values
fitted_values = double_sigmoid(GroupData_morning["end_time"], L1, k1, x01, L2, k2, x02)

# Calculate residuals
residuals = GroupData_morning["cumulative_solved_trials"] - fitted_values

# Calculate SS_res and SS_tot
SS_res = np.sum(residuals**2)
SS_tot = np.sum(
    (
        GroupData_morning["cumulative_solved_trials"]
        - np.mean(GroupData_morning["cumulative_solved_trials"])
    )
    ** 2
)

# Calculate R-squared
r_squared = 1 - (SS_res / SS_tot)
print(f"R-squared: {r_squared}")

# Create a new DataFrame for the double sigmoid fit curve
double_sigmoid_fit = pd.DataFrame(
    {
        "end_time": np.linspace(
            GroupData_morning["end_time"].min(),
            GroupData_morning["end_time"].max(),
            100,
        ),
    }
)

# Calculate y values for the double sigmoid fit curve
double_sigmoid_fit["cumulative_solved_trials"] = double_sigmoid(
    double_sigmoid_fit["end_time"], L1, k1, x01, L2, k2, x02
)

# Plot the original data and the double sigmoid fit curve
cumulcurve_pool = hv.Curve(
    data=GroupData_morning,
    kdims=["end_time"],
    vdims=["cumulative_solved_trials"],
).opts(
    height=1000,
    width=1200,
    alpha=1,
    line_width=2,
    xlabel="Time(s)",
    ylabel="Cumulative Solved Trials",
    show_grid=True,
    fontscale=3,
    title="Cumulative Solved Trials Over Time",
) * hv.Curve(
    data=double_sigmoid_fit, kdims=["end_time"], vdims=["cumulative_solved_trials"]
).opts(
    color="green"
)

cumulcurve_pool

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy import stats
import holoviews as hv
import hvplot.pandas
from holoviews import opts  # Import opts from holoviews


# Define logistic (sigmoid) function
def sigmoid(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))


# Filter data to include only relevant subset
GroupData_morning = trial_end_times

# Calculate the number of unique 'flies' in the dataset
num_replicates = GroupData_morning["fly"].nunique()

# Initial guesses for L, k, x0
p0 = [
    max(GroupData_morning["cumulative_solved_trials"]),
    1,
    np.median(GroupData_morning["end_time"]),
]

# Fit sigmoid function to data
params, _ = curve_fit(
    sigmoid,
    GroupData_morning["end_time"],
    GroupData_morning["cumulative_solved_trials"],
    p0,
    maxfev=5000,
)

# params contains the fitted values for L, k, x0
L, k, x0 = params

# Create a new DataFrame for the sigmoid fit curve
sigmoid_fit = pd.DataFrame(
    {
        "end_time": np.linspace(
            GroupData_morning["end_time"].min(),
            GroupData_morning["end_time"].max(),
            100,
        ),
    }
)

# Calculate y values for the sigmoid fit curve
sigmoid_fit["cumulative_solved_trials"] = sigmoid(sigmoid_fit["end_time"], L, k, x0)

# Calculate R-squared for sigmoid fit curve
y_pred_sigmoid = sigmoid(GroupData_morning["end_time"], L, k, x0)
r_squared_sigmoid = r2_score(
    GroupData_morning["cumulative_solved_trials"], y_pred_sigmoid
)

# Calculate linear fit for the first half of the data
first_half = GroupData_morning.iloc[: len(GroupData_morning) // 2]
slope, intercept, _, _, _ = stats.linregress(
    first_half["end_time"], first_half["cumulative_solved_trials"]
)

# Create a new DataFrame for the linear fit line
linear_fit = pd.DataFrame(
    {
        "end_time": np.linspace(
            GroupData_morning["end_time"].min(),
            GroupData_morning["end_time"].max(),
            100,
        ),
    }
)

# Calculate y values for the linear fit line
linear_fit["cumulative_solved_trials"] = slope * linear_fit["end_time"] + intercept

# Calculate R-squared for linear fit curve
y_pred_linear = slope * GroupData_morning["end_time"] + intercept
r_squared_linear = r2_score(
    GroupData_morning["cumulative_solved_trials"], y_pred_linear
)

# Calculate center x value and min and maximum xy value
center_x = (
    GroupData_morning["end_time"].max() - GroupData_morning["end_time"].min()
) / 2
max_y = max(
    max(GroupData_morning["cumulative_solved_trials"]),
    max(sigmoid_fit["cumulative_solved_trials"]),
)

max_x = GroupData_morning["end_time"].max()
min_y = GroupData_morning["cumulative_solved_trials"].min()

# Create your plot using GroupData_morning and add the fits and annotations
cumulcurve_pool = (
    hv.Curve(
        data=GroupData_morning,
        kdims=["end_time"],
        vdims=["cumulative_solved_trials"],
    )
    .opts(
        height=1000,
        width=1200,
        alpha=1,
        line_width=2,
        xlabel="Time(s)",
        ylabel="Cumulative Solved Trials",
        show_grid=True,
        fontscale=3,
        title="Cumulative Solved Trials Over Time",
    )
    * hv.Curve(
        data=sigmoid_fit, kdims=["end_time"], vdims=["cumulative_solved_trials"]
    ).opts(color="blue")
    * hv.Curve(
        data=linear_fit, kdims=["end_time"], vdims=["cumulative_solved_trials"]
    ).opts(color="red")
    # Uncomment the following lines to add text annotations
    * hv.Text(center_x, max_y - 10, f"Sigmoid fit R-squared: {r_squared_sigmoid:.2f}").opts(text_color="blue")
    * hv.Text(center_x, max_y, f"Linear fit R-squared: {r_squared_linear:.2f}").opts(text_color="red")
    * hv.Text(max_x - 100, min_y * num_replicates, f"N = {num_replicates}")
).opts(
    opts.Area(
        show_legend=False,
        show_frame=False,
        fill_color="blue",
        line_color="black",
        line_width=0,
    )
)

cumulcurve_pool